In [1]:
import os
os.getcwd()
os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\sleigh_the_model


In [38]:
import pandas as pd
import scripts.features as ft
import scripts.models as md
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, Lasso
import numpy as np

In [3]:
file = 'data/dataformodel.csv'

In [4]:
months_in_adv = 12

In [5]:
data = ft.load_data(file, months_in_adv)

In [6]:
features_to_lag = [
    'pce',
    'inflation',
    'unemployment_rate',
    'revolving_credit',
    'nonrevolving_credit',
    'total_credit'
]

In [7]:
data_with_lag = ft.add_lag_features(data, months_in_adv, features_to_lag)

In [8]:
data_with_lag = data_with_lag.dropna()

In [9]:
X = data_with_lag.drop(columns=['date', 'pce_future'])
y = data_with_lag['pce_future']

In [10]:
test_size = 60
min_train_size = 60
total_months = len(data_with_lag)
max_splits = (total_months - min_train_size) // test_size
print(max_splits)
n_splits = max_splits

11


In [11]:
time_series_split = TimeSeriesSplit(
    n_splits = n_splits,
    test_size = test_size
)

In [12]:
lg = LinearRegression() 

In [13]:
lr_mse_scores, lr_r2_scores, lr_test_results = md.train_eval_model(X, y, lg, time_series_split, data_with_lag, 'Linear Regression')

*** Linear Regression ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  492.92
    R-squared:  0.96
    Actual PCE: 995.00
    Predicted PCE: 967.71

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  3661.26
    R-squared:  0.91
    Actual PCE: 1695.20
    Predicted PCE: 1600.48

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  16245.63
    R-squared:  0.78
    Actual PCE: 2659.50
    Predicted PCE: 2824.94

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  51976.04
    R-squared:  0.51
    Actual PCE: 3770.00
    Predicted PCE: 4011.96

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05 to 199

From looking at the metrics the Linear Regression model preformed second best. Tried Lasso after working on forcasting notebook and that appears to outperform linear regression model 

In [14]:
xgbr = xgb.XGBRegressor(
    objective = 'reg:squarederror', 
    n_estimators = 100, 
    learning_rate = 0.1, 
    max_depth = 6, 
    random_state = 42
) 

In [15]:
xgbr_mse_scores, xgbr_r2_scores, xgbr_test_results = md.train_eval_model(X, y, xgbr, time_series_split, data_with_lag, 'XGB Regressor')

*** XGB Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  39572.23
    R-squared: -2.39
    Actual PCE: 995.00
    Predicted PCE: 635.54

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  162380.80
    R-squared: -2.78
    Actual PCE: 1695.20
    Predicted PCE: 994.00

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  293219.90
    R-squared: -2.92
    Actual PCE: 2659.50
    Predicted PCE: 1700.98

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  390306.46
    R-squared: -2.71
    Actual PCE: 3770.00
    Predicted PCE: 2656.21

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05 to 19

Looking at metrics XGB Regressor model did not out preform Linear Regression  

In [16]:
rfr = RandomForestRegressor(
    random_state=42
)

In [17]:
rfr_mse_scores, rfr_r2_scores, rfr_test_results = md.train_eval_model(X, y, rfr, time_series_split, data_with_lag, 'Random Forest Regressor')

*** Random Forest Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  49092.26
    R-squared: -3.21
    Actual PCE: 995.00
    Predicted PCE: 609.20

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  194665.00
    R-squared: -3.53
    Actual PCE: 1695.20
    Predicted PCE: 949.20

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  384751.82
    R-squared: -4.15
    Actual PCE: 2659.50
    Predicted PCE: 1599.23

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  513329.93
    R-squared: -3.87
    Actual PCE: 3770.00
    Predicted PCE: 2549.23

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (198

Looking at metrics Random Forest Regressor model did not out preform Linear Regression  

In [18]:
cat = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=0
)

In [19]:
cat_mse_scores, cat_r2_scores, cat_test_results = md.train_eval_model(X, y, cat, time_series_split, data_with_lag, 'Cat Boost Regressor')

*** Cat Boost Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  65218.32
    R-squared: -4.59
    Actual PCE: 995.00
    Predicted PCE: 568.05

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  201135.95
    R-squared: -3.68
    Actual PCE: 1695.20
    Predicted PCE: 959.78

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  409926.66
    R-squared: -4.48
    Actual PCE: 2659.50
    Predicted PCE: 1562.28

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  689476.45
    R-squared: -5.55
    Actual PCE: 3770.00
    Predicted PCE: 2347.54

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05

Looking at metrics Cat Boost Regressor model did not out preform Linear Regression 

In [20]:
lgbm = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

Tried lgbm, there was multiple warnings of no further splits with positive gain 

baseline model 

In [21]:
baseline = ft.load_data(file, months_in_adv)

In [22]:
baseline

,date,pce,inflation,unemployment_rate,revolving_credit,nonrevolving_credit,total_credit,pce_future
0,1959-01-01,306.1,29.010,6.0,0.00,48961.16,48961.16,323.6
1,1959-02-01,309.6,29.000,5.9,0.00,49513.71,49513.71,325.3
2,1959-03-01,312.7,28.970,5.6,0.00,50007.73,50007.73,330.2
3,1959-04-01,312.2,28.980,5.2,0.00,50463.43,50463.43,336.5
4,1959-05-01,316.1,29.040,5.1,0.00,51007.24,51007.24,330.0
...,...,...,...,...,...,...,...,...
791,2024-12-01,20408.1,317.603,4.1,1316865.89,3671015.14,4987881.03,NaN
792,2025-01-01,20389.0,319.086,4.0,1323514.25,3673358.72,4996872.97,NaN
793,2025-02-01,20469.3,319.775,4.1,1323945.56,3672057.95,4996003.50,NaN
794,2025-03-01,20621.8,319.615,4.2,1313730.92,3678871.89,4992602.81,NaN


In [23]:
baseline['pce_base_pred'] = baseline['pce'].shift(12)

In [24]:
baseline_df = baseline.dropna(subset=['pce_future', 'pce_base_pred']).copy()

actual_values = baseline_df['pce_future']
predicted_values = baseline_df['pce_base_pred']

print(baseline_df[['date', 'pce', 'pce_future', 'pce_base_pred']].head())

         date    pce  pce_future  pce_base_pred
12 1960-01-01  323.6       332.2          306.1
13 1960-02-01  325.3       333.7          309.6
14 1960-03-01  330.2       337.7          312.7
15 1960-04-01  336.5       337.6          312.2
16 1960-05-01  330.0       339.9          316.1


In [25]:
mse = mean_squared_error(actual_values, predicted_values)
print(f"\nSeasonal Naive Baseline - Mean Squared Error (MSE): {mse:.2f}")

# Calculate R-squared (R2)
r2 = r2_score(actual_values, predicted_values)
print(f"Seasonal Naive Baseline - R-squared (R2): {r2:.2f}")


Seasonal Naive Baseline - Mean Squared Error (MSE): 762718.78
Seasonal Naive Baseline - R-squared (R2): 0.97


trying to tune xgboost, 

In [26]:
xgb_tune = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=150,
    learning_rate=0.05, 
    max_depth=5,   
    subsample=0.7, 
    colsample_bytree=0.7,    
    random_state=42   
)

In [27]:
xgb_tune_mse_scores, xgb_tune_r2_scores, xgb_tune_test_results = md.train_eval_model(X, y, xgb_tune, time_series_split, data_with_lag, 'Tuned XGB Regressor')

*** Tuned XGB Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  42467.85
    R-squared: -2.64
    Actual PCE: 995.00
    Predicted PCE: 625.97

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  167125.48
    R-squared: -2.89
    Actual PCE: 1695.20
    Predicted PCE: 987.16

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  303669.44
    R-squared: -3.06
    Actual PCE: 2659.50
    Predicted PCE: 1689.89

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  407568.81
    R-squared: -2.87
    Actual PCE: 3770.00
    Predicted PCE: 2640.23

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05

Looking at metrics Ridge model did not out preform Linear Regression 

In [31]:
ridge = Ridge(alpha=1.0)

In [32]:
ridge_mse_scores, ridge_r2_scores, ridge_test_results = md.train_eval_model(X, y, ridge, time_series_split, data_with_lag, 'Random Forest Regressor')

*** Random Forest Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  421.97
    R-squared:  0.96
    Actual PCE: 995.00
    Predicted PCE: 982.06

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  2609.63
    R-squared:  0.94
    Actual PCE: 1695.20
    Predicted PCE: 1629.90

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  11187.73
    R-squared:  0.85
    Actual PCE: 2659.50
    Predicted PCE: 2788.51

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  47757.14
    R-squared:  0.55
    Actual PCE: 3770.00
    Predicted PCE: 4013.93

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05 

Lasso model appears to out preform the linear regression model 

In [35]:
lasso =  Lasso(alpha=0.1, max_iter=10000)

In [36]:
lasso_mse_scores, lasso_r2_scores, lasso_test_results = md.train_eval_model(X, y, lasso, time_series_split, data_with_lag, 'Random Forest Regressor')

*** Random Forest Regressor ***

--- Split 1 ---
  TRAIN: 112 months (1960-01 to 1969-04)
  TEST: 60 months (1969-05 to 1974-04)
Model Performance:
    Mean Squared Error:  441.21
    R-squared:  0.96
    Actual PCE: 995.00
    Predicted PCE: 1013.03

--- Split 2 ---
  TRAIN: 172 months (1960-01 to 1974-04)
  TEST: 60 months (1974-05 to 1979-04)
Model Performance:
    Mean Squared Error:  720.44
    R-squared:  0.98
    Actual PCE: 1695.20
    Predicted PCE: 1686.74

--- Split 3 ---
  TRAIN: 232 months (1960-01 to 1979-04)
  TEST: 60 months (1979-05 to 1984-04)
Model Performance:
    Mean Squared Error:  7925.04
    R-squared:  0.89
    Actual PCE: 2659.50
    Predicted PCE: 2773.71

--- Split 4 ---
  TRAIN: 292 months (1960-01 to 1984-04)
  TEST: 60 months (1984-05 to 1989-04)
Model Performance:
    Mean Squared Error:  23348.68
    R-squared:  0.78
    Actual PCE: 3770.00
    Predicted PCE: 3951.58

--- Split 5 ---
  TRAIN: 352 months (1960-01 to 1989-04)
  TEST: 60 months (1989-05 t

In [39]:
np.mean(lasso_r2_scores)

0.8290706499548975

In [40]:
np.mean(lr_r2_scores)

0.7347335882067046